In [1]:
import os
import sys
import nengo
import keras
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import nengo_dl
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split, ShuffleSplit
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Dropout, AveragePooling2D, Flatten, Dense

In [2]:
# global config
dataset_path = os.path.join('..', 'datasets', 'VarekaGTNEpochs.mat')
os.makedirs('nengo', exist_ok=True)
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
def get_dataset(file = dataset_path):
    """
    Helper function to get dataset from file
    """

    np_file = loadmat(file) # load dataset file with matrices
    target_data = np_file['allTargetData'] # get target data
    non_target_data = np_file['allNonTargetData'] # get non target data

    features = np.concatenate((target_data, non_target_data)) # concatenate dataset as features

    # target labels are represented as (1, 0) vector, non target labels are represented as (0, 1) vector
    target_labels = np.tile(np.array([1, 0]), (target_data.shape[0], 1)) # set 'target labels' as (1, 0) vector
    non_target_labels = np.tile(np.array([0, 1]), (non_target_data.shape[0], 1)) # set 'non target' as (0, 1) vector
    labels = np.vstack((target_labels, non_target_labels)) # concatenate target and non target labels

    # filter noise above 100 mV
    threshold = 100.0
    result_x, result_y = [], []
    for i in range(features.shape[0]):
        if not np.max(np.abs(features[i])) > threshold:
            result_x.append(features[i])
            result_y.append(labels[i])

    features, labels = np.array(result_x), np.array(result_y)
    features = features.reshape((features.shape[0], 1, -1))
    labels = labels.reshape((labels.shape[0], 1, -1))
    print('features:', features, features.shape)
    print('labels:', labels, labels.shape)
    return features, labels

In [4]:
def create_model():
    """
    Function to create tensorflow model
    """

    # Some layers are replaced by tensornodes might be problem or not
    input = Input(shape=(1200, 3, 1), name='input_layer')
    conv2d = Conv2D(filters=6, kernel_size=(3, 3), activation=tf.nn.elu)(input)
    dropout1 = Dropout(0.5, seed=0)(conv2d)
    # padding must be same in order to compile
    avg_pooling = AveragePooling2D(pool_size=(1, 8), padding='same')(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(100, activation=tf.nn.elu)(flatten)
    batch_norm = BatchNormalization()(dense1)
    dropout2 = Dropout(0.5, seed=0)(batch_norm)
    output = Dense(2, activation=tf.nn.sigmoid, name='output_layer')(dropout2)

    return Model(inputs=input, outputs=output)

In [5]:
def run_nengo_ann(train_x, train_y, valid_x, valid_y, test_x, test_y,
                  param_output=os.path.join('nengo', 'network_params')):
    """
    Runs ann network with nengo wrapper
    """
    model = create_model()
    converter = nengo_dl.Converter(model)

    early_stop = EarlyStopping(patience=5, verbose=1, restore_best_weights=True)

    # run ann with nengo
    with nengo_dl.Simulator(converter.net, minibatch_size=16) as simulator:
        simulator.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        simulator.fit(x=train_x, y=train_y,
                                validation_data=(valid_x, valid_y),
                                epochs=30,
                                shuffle=True,  # delete later to see if it makes any difference
                                callbacks=[early_stop]
                                )

        ann_results = simulator.evaluate(x=test_x, y=test_y)
        simulator.save_params(param_output)

    return model, ann_results

def run_nengo_spiking(model, test_x, test_y, params=os.path.join('nengo', 'network_params'),
                      spiking_scaling=1000, synapse=0.01, timesteps = 30):
    """
    Runs SNN as ANN parameter with SpikingRectifiedLinear activation instead of relu
    """
    converter = nengo_dl.Converter(model=model,
                                   swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                                   scale_firing_rates=spiking_scaling, synapse=synapse)

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    output_layer = converter.outputs[model.get_layer('output_layer')]

    time_tiled_x = np.tile(test_x, (1, timesteps, 1))

    with nengo_dl.Simulator(converter.net, minibatch_size=41, progress_bar=False) as simulator:
        simulator.load_params(params)
        predictions = simulator.predict(time_tiled_x)[output_layer]
        # slice all except for last timestep and get argmax of it
        predictions = predictions[:, timesteps-2, :]

        predictions = np.argmax(predictions, axis=-1)

        transformed_test_y = np.squeeze(test_y, axis=1)
        transformed_test_y = np.argmax(test_y, axis=-1)
        avg = (predictions == transformed_test_y).mean()
        keras.backend.clear_session()
        gc.collect()

    del transformed_test_y, time_tiled_x
    return avg

In [6]:
# Driver code
features, labels = get_dataset()
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=.25, random_state=seed, shuffle=True)

print('train_x: {}, shape: {}, train_y: {}, shape: {}'.format(train_x, train_x.shape, train_y, train_y.shape))
print('test_x: {}, shape: {}, test_y: {}, shape: {}'.format(test_x, test_x.shape, test_y, test_y.shape))

ann, snn = [], []

iter = 1
monte_carlo = ShuffleSplit(n_splits=30, test_size=.25, random_state=seed)
for train, validation in monte_carlo.split(train_x):

    print('iteration {}'.format(iter))
    print('train: {}, validation: {}'.format(train, validation))

    curr_train_x, curr_train_y = train_x[train], train_y[train]
    curr_valid_x, curr_valid_y = train_x[validation], train_y[validation]

    params_output = os.path.join('nengo','params_iter_{}'.format(iter))
    model, ann_eval = run_nengo_ann(
        train_x=curr_train_x, train_y=curr_train_y, valid_x=curr_valid_x, valid_y=curr_valid_y,
        test_x=test_x, test_y=test_y, param_output=params_output
    )
    print('{}. ann accuracy {:5f}%'.format(iter, ann_eval['probe_accuracy'] * 100))

    ann.append(ann_eval['probe_accuracy'])

    snn_eval = run_nengo_spiking(test_x=test_x, test_y=test_y, model=model, params=params_output)

    snn.append(snn_eval)
    print('{}. snn accuracy {:5f}%'.format(iter, snn_eval * 100))

    del curr_train_x, curr_train_y, curr_valid_x, curr_valid_y
    keras.backend.clear_session() # clear session to not run out of memory eventually
    iter += 1

features: [[[ -0.77055806  -0.76216829  -0.75329262 ...   0.9730792    1.04828787
     1.00901747]]

 [[  4.72857332   4.79228735   4.86664867 ... -15.98517323 -16.30454826
   -16.64167595]]

 [[ -2.2145133   -1.49968874  -0.77021754 ...  -6.05920124  -5.69483948
    -5.42987347]]

 ...

 [[  9.08650398   9.06958485   9.0740099  ...  18.27712059  17.41079521
    16.49032021]]

 [[  3.50287008   2.95584893   2.36093497 ...  20.93103218  21.30741882
    21.62874603]]

 [[-14.8089571  -14.57761955 -14.33197689 ...  32.55890274  32.86030197
    33.12610626]]] (8036, 1, 3600)
labels: [[[1 0]]

 [[1 0]]

 [[1 0]]

 ...

 [[0 1]]

 [[0 1]]

 [[0 1]]] (8036, 1, 2)
train_x: [[[ -0.5420959   -1.21452522  -1.89456093 ...  -1.16598654  -1.73066759
    -2.2462945 ]]

 [[ -9.35857487  -9.43084145  -9.40663052 ...  -1.80052102  -0.70815951
     0.17630903]]

 [[  4.39046383   3.40499306   2.36262703 ...  38.03783417  38.5178566
    38.89231491]]

 ...

 [[-19.15268135 -19.42535782 -19.74151993 ... -1

c:\users\itznu\anaconda3\envs\tensorflow-gpu\lib\site-packages\nengo_dl\converter.py:589: UserWarning: Activation type <function elu at 0x0000027CDA59F310> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(
c:\users\itznu\anaconda3\envs\tensorflow-gpu\lib\site-packages\nengo_dl\converter.py:325: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\users\itznu\anaconda3\envs\tensorflow-gpu\lib\site-packages\nengo_dl\converter.py:325: UserWarning: average_pooling2d.padding has value same != valid, which is not supported. Falling back to TensorNode.
  warnings.warn(
c:\users\itznu\anaconda3\envs\tensorflow-gpu\lib\site-packages\nengo_dl\converter.py:325: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True or layer.trainable=False. Falling back to TensorNode.
  warnings.warn(
c:\users\itznu\ana

KeyboardInterrupt: 

In [ ]:
# Visualize using bar graph

def visualize(ann_values, snn_values):

    data = {'ann_accuracy': ann_values, 'snn_accuracy': snn_values}
    df = pd.DataFrame(data=data, index=(list(range(1, len(ann_values) + 1))))

    print(df)
    df.to_excel(os.path.join('output','values.xlsx'))

    # for i in range(len(snn_eval)):
    #     print('Iteration {}. ANN Accuracy: {:5f}%, SNN Accuracy: {:5f}%'.format(
    #         i+1, ann_eval[i]['probe_accuracy'] * 100, snn_eval * 100
    #     ))

    # fig = plt.figure()
    #
    # average_ann, average_snn = [], snn_eval
    # for eval in ann_eval:
    #     average_ann.append(eval['probe_accuracy'])
    #
    # fig, ax = plt.subplots()
    # index = np.arange(len(average_ann))
    #
    # bar_width = 0.3
    # ann_bars = plt.bar(index, average_ann, bar_width, color='orange', label='ANN')
    # snn_bars = plt.bar(index + bar_width, average_snn, bar_width, color='cornflowerblue', label='SNN')
    #
    # def make_labels(rects):
    #     for rect in rects:
    #         height = rect.get_height()
    #         ax.text(rect.get_x() + rect.get_width() / 2, height,
    #                 '{}'.format(height),
    #                 ha='center', va='bottom')
    #
    # make_labels(ann_bars)
    #
    # plt.xlabel('Iteration')
    # plt.ylabel('Accuracy')
    # plt.title('Average test accuracy per Shuffle and Split iteration')
    # plt.legend()
    # plt.tight_layout()
    #
    # plt.show()
    #
    # os.makedirs('output', exist_ok=True)
    # plt.savefig(os.path.join('output', 'acc.svg'))

visualize(ann, snn)